In [ ]:
# Load the MNIST dataset and preprocess it for GAN training.

from tensorflow.keras.datasets import mnist
import numpy as np

# Load dataset
(X_train, _), (_, _) = mnist.load_data()

# Normalize images to the range [0, 1] and reshape
X_train = X_train / 255.0
X_train = np.expand_dims(X_train, axis=-1)

print("Training data shape:", X_train.shape)


In [ ]:
# Load the MNIST dataset and preprocess it for GAN training.

from tensorflow.keras.datasets import mnist
import numpy as np

# Load dataset
(X_train, _), (_, _) = mnist.load_data()

# Normalize images to the range [0, 1] and reshape
X_train = X_train / 255.0
X_train = np.expand_dims(X_train, axis=-1)

print("Training data shape:", X_train.shape)


In [ ]:
# Define the Generator model to create synthetic images from random noise.

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, BatchNormalization, LeakyReLU, Conv2DTranspose

def build_generator(latent_dim):
    model = Sequential([
        Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim),
        Reshape((7, 7, 128)),
        BatchNormalization(),
        Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same", activation="relu"),
        BatchNormalization(),
        Conv2DTranspose(64, (4, 4), strides=(2, 2), padding="same", activation="relu"),
        BatchNormalization(),
        Conv2DTranspose(1, (4, 4), padding="same", activation="sigmoid")
    ])
    return model

latent_dim = 100
generator = build_generator(latent_dim)
generator.summary()


In [ ]:
# Define the Discriminator model to classify images as real or fake.

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout

def build_discriminator(input_shape):
    model = Sequential([
        Conv2D(64, (4, 4), strides=(2, 2), padding="same", activation=LeakyReLU(0.2), input_shape=input_shape),
        Dropout(0.3),
        Conv2D(128, (4, 4), strides=(2, 2), padding="same", activation=LeakyReLU(0.2)),
        Dropout(0.3),
        Flatten(),
        Dense(1, activation="sigmoid")
    ])
    return model

input_shape = (28, 28, 1)
discriminator = build_discriminator(input_shape)
discriminator.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
discriminator.summary()


In [ ]:
# Combine the Generator and Discriminator to form the GAN model.

from tensorflow.keras.models import Sequential

def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze Discriminator during GAN training
    model = Sequential([generator, discriminator])
    model.compile(optimizer="adam", loss="binary_crossentropy")
    return model

gan = build_gan(generator, discriminator)
gan.summary()


In [ ]:
# Train the GAN using alternating updates for the Generator and Discriminator.

import matplotlib.pyplot as plt

def train_gan(generator, discriminator, gan, X_train, latent_dim, epochs, batch_size):
    for epoch in range(epochs):
        # Train the Discriminator
        idx = np.random.randint(0, X_train.shape[0], batch_size // 2)
        real_images = X_train[idx]
        fake_images = generator.predict(np.random.randn(batch_size // 2, latent_dim))
        
        real_labels = np.ones((batch_size // 2, 1))
        fake_labels = np.zeros((batch_size // 2, 1))
        
        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # Train the Generator
        noise = np.random.randn(batch_size, latent_dim)
        valid_labels = np.ones((batch_size, 1))  # Wants to fool the discriminator
        g_loss = gan.train_on_batch(noise, valid_labels)
        
        # Print progress
        if (epoch + 1) % 100 == 0:
            print(f"Epoch {epoch + 1}/{epochs} | D Loss: {d_loss[0]:.4f} | G Loss: {g_loss:.4f}")
            
            # Generate and save sample images
            save_generated_images(generator, epoch, latent_dim)

def save_generated_images(generator, epoch, latent_dim, examples=10, dim=(1, 10)):
    noise = np.random.randn(examples, latent_dim)
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(-1, 28, 28)
    
    plt.figure(figsize=(10, 2))
    for i in range(examples):
        plt.subplot(dim[0], dim[1], i + 1)
        plt.imshow(generated_images[i], cmap="gray")
        plt.axis("off")
    plt.tight_layout()
    plt.savefig(f"generated_images_epoch_{epoch}.png")
    plt.show()

# Train the GAN
train_gan(generator, discriminator, gan, X_train, latent_dim, epochs=5000, batch_size=64)
